In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'sample_submission.csv']


In [2]:
os.listdir("../input/train")

['testing_list.txt', 'audio', 'validation_list.txt', 'LICENSE', 'README.md']

In [3]:
test_list=pd.read_csv("../input/train/testing_list.txt",sep=" ", header=None)
val_list=pd.read_csv("../input/train/validation_list.txt",sep=" ", header=None)

In [4]:
len(val_list),len(test_list)

(6798, 6835)

In [5]:
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import gc
from scipy.io import wavfile
import librosa
import IPython.display as ipd
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
train_data_path = "../input/train/audio"
#test_data_path = "../input/test/audio"

In [7]:
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [8]:
labels, fnames = list_wavs_fname(train_data_path)

../input/train/audio


In [9]:
len(labels),len(fnames)

(64727, 64727)

In [10]:
val=[]
for i in range(len(val_list)):
    val.append(val_list[0][i])
len(val)

6798

In [11]:
test=[]
for i in range(len(test_list)):
    test.append(test_list[0][i])
len(test)

6835

In [12]:
L=16000
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))

def chop_audio(samples, L=16000, num=20):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]

def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [13]:
np.unique(labels)

array(['_background_noise_', 'bed', 'bird', 'cat', 'dog', 'down', 'eight',
       'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine',
       'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six',
       'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero'],
      dtype='<U18')

In [14]:
labels[0]+"/"+fnames[0] in test

True

In [15]:
x=[]
y=[]

for label, fname in zip(labels, fnames):
    #print(label,fname)
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples= pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else :
        n_samples = [samples]
    for samples in n_samples:
        samples=np.asarray(samples).astype('float')
        S = librosa.feature.melspectrogram(samples, sr=sample_rate, n_mels=128)
        log_S = librosa.power_to_db(S, ref=np.max)
        mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
        x.append(mfcc)
        y.append(label)
        
     

/opt/conda/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [16]:
len(x),len(y)

(64841, 64841)

In [17]:
y_ = label_transform(y)

In [18]:
y_.shape

(64841, 12)

In [23]:
y_=y_.values

In [24]:
y_[0]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [26]:
y_train = []
x_train = []
x_val=[]
y_val=[]
x_test=[]
y_test=[]
for i in range(len(labels)):
    if(labels[i]+'/'+fnames[i] in val):
        y_val.append(y_[i])
        x_val.append(x[i])
    elif(labels[i]+'/'+fnames[i] in test):
        y_test.append(y_[i])
        x_test.append(x[i])
    else:
        y_train.append(y_[i])
        x_train.append(x[i])
    

In [27]:
x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)
print(x_train.shape,x_test.shape,x_val.shape)

(51094, 13, 32) (6835, 13, 32) (6798, 13, 32)


In [28]:
x_train=x_train.reshape(tuple(list(x_train.shape) + [1]))
x_test=x_test.reshape(tuple(list(x_test.shape) + [1]))
x_val=x_val.reshape(tuple(list(x_val.shape) + [1]))
print(x_train.shape,x_test.shape,x_val.shape)

(51094, 13, 32, 1) (6835, 13, 32, 1) (6798, 13, 32, 1)


In [30]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
print(y_train.shape,y_test.shape,y_val.shape)

(51094, 12) (6835, 12) (6798, 12)


In [ ]:
# y_train = label_transform(y_train)
# y_test = label_transform(y_test)
# y_val = label_transform(y_val)

In [ ]:
# label_index = y_train.columns.values
# y_train = y_train.values
# y_train = np.array(y_train)
# y_test = y_test.values
# y_test = np.array(y_test)
# y_val = y_val.values
# y_val = np.array(y_val)
# print(y_train.shape,y_test.shape,y_val.shape)

In [31]:
from keras.models import Model
from keras.layers import Dense,Input,Conv2D,MaxPooling2D,GlobalAveragePooling2D,Activation,Dropout,Reshape,concatenate,BatchNormalization
from keras.losses import binary_crossentropy,categorical_crossentropy
from keras.optimizers import Adam,SGD,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras import backend as K 

In [32]:
def make_model():
    inputs=Input(shape=(13,32,1))
    l=inputs
    l=Conv2D(filters=8,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=8,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    
    l=Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    l=Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=48,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    
    l=GlobalAveragePooling2D()(l)
    l=Dropout(0.4)(l)
    l=Dense(64,activation='relu')(l)
    l=Dropout(0.4)(l)
    l=Dense(64,activation='relu')(l)
    l=Dense(12,activation='softmax')(l)
    outputs=l
    return inputs,outputs

In [33]:
inputs,outputs=make_model()
model=Model(inputs=inputs,outputs=outputs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 32, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 32, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 13, 32, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 32, 8)         584       
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 32, 8)         32        
_________________________________________________________________
activation_2 (Activation)    (None, 13, 32, 8)         0         
__________

In [34]:
optim=Adam(1e-4)#optim=SGD(1e-3)
loss=categorical_crossentropy
model.compile(optim,loss,metrics=['accuracy'])

In [35]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=32,verbose=1)

Train on 51094 samples, validate on 6798 samples
Epoch 1/10
51094/51094 [==============================] - 31s 611us/step - loss: 1.5265 - acc: 0.6119 - val_loss: 1.3026 - val_acc: 0.6196
Epoch 2/10
51094/51094 [==============================] - 29s 570us/step - loss: 1.2505 - acc: 0.6386 - val_loss: 1.0992 - val_acc: 0.6534
Epoch 3/10
51094/51094 [==============================] - 29s 571us/step - loss: 1.0687 - acc: 0.6658 - val_loss: 0.8698 - val_acc: 0.7127
Epoch 4/10
51094/51094 [==============================] - 29s 567us/step - loss: 0.8985 - acc: 0.7041 - val_loss: 0.6872 - val_acc: 0.7614
Epoch 5/10
51094/51094 [==============================] - 29s 570us/step - loss: 0.7613 - acc: 0.7437 - val_loss: 0.5851 - val_acc: 0.8191
Epoch 6/10
51094/51094 [==============================] - 29s 571us/step - loss: 0.6688 - acc: 0.7771 - val_loss: 0.5179 - val_acc: 0.8350
Epoch 7/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.6027 - acc: 0.7994 - val_loss: 0.4

In [36]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=32,verbose=1)

Train on 51094 samples, validate on 6798 samples
Epoch 1/10
51094/51094 [==============================] - 29s 566us/step - loss: 0.4638 - acc: 0.8478 - val_loss: 0.3802 - val_acc: 0.8842
Epoch 2/10
51094/51094 [==============================] - 29s 567us/step - loss: 0.4423 - acc: 0.8567 - val_loss: 0.3971 - val_acc: 0.8739
Epoch 3/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.4224 - acc: 0.8642 - val_loss: 0.3399 - val_acc: 0.8960
Epoch 4/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.4034 - acc: 0.8713 - val_loss: 0.3534 - val_acc: 0.8926
Epoch 5/10
51094/51094 [==============================] - 29s 570us/step - loss: 0.3883 - acc: 0.8765 - val_loss: 0.3264 - val_acc: 0.8938
Epoch 6/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.3712 - acc: 0.8824 - val_loss: 0.3249 - val_acc: 0.8978
Epoch 7/10
51094/51094 [==============================] - 29s 569us/step - loss: 0.3593 - acc: 0.8857 - val_loss: 0.3

In [37]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=32,verbose=1)

Train on 51094 samples, validate on 6798 samples
Epoch 1/10
51094/51094 [==============================] - 29s 569us/step - loss: 0.3195 - acc: 0.8993 - val_loss: 0.2862 - val_acc: 0.9112
Epoch 2/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.3071 - acc: 0.9028 - val_loss: 0.2891 - val_acc: 0.9100
Epoch 3/10
51094/51094 [==============================] - 29s 567us/step - loss: 0.3021 - acc: 0.9054 - val_loss: 0.2771 - val_acc: 0.9154
Epoch 4/10
51094/51094 [==============================] - 29s 566us/step - loss: 0.2935 - acc: 0.9092 - val_loss: 0.2824 - val_acc: 0.9110
Epoch 5/10
51094/51094 [==============================] - 29s 565us/step - loss: 0.2853 - acc: 0.9111 - val_loss: 0.2852 - val_acc: 0.9192
Epoch 6/10
51094/51094 [==============================] - 29s 575us/step - loss: 0.2791 - acc: 0.9133 - val_loss: 0.2669 - val_acc: 0.9200
Epoch 7/10
51094/51094 [==============================] - 29s 573us/step - loss: 0.2744 - acc: 0.9153 - val_loss: 0.2

In [38]:
K.set_value(model.optimizer.lr,1e-5)

In [39]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=32,verbose=1)

Train on 51094 samples, validate on 6798 samples
Epoch 1/10
51094/51094 [==============================] - 29s 566us/step - loss: 0.2361 - acc: 0.9274 - val_loss: 0.2454 - val_acc: 0.9269
Epoch 2/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.2307 - acc: 0.9283 - val_loss: 0.2459 - val_acc: 0.9275
Epoch 3/10
51094/51094 [==============================] - 29s 569us/step - loss: 0.2288 - acc: 0.9291 - val_loss: 0.2462 - val_acc: 0.9284
Epoch 4/10
51094/51094 [==============================] - 29s 567us/step - loss: 0.2278 - acc: 0.9305 - val_loss: 0.2472 - val_acc: 0.9282
Epoch 5/10
51094/51094 [==============================] - 29s 567us/step - loss: 0.2272 - acc: 0.9300 - val_loss: 0.2430 - val_acc: 0.9285
Epoch 6/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.2267 - acc: 0.9301 - val_loss: 0.2459 - val_acc: 0.9285
Epoch 7/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.2285 - acc: 0.9295 - val_loss: 0.2

In [40]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=32,verbose=1)

Train on 51094 samples, validate on 6798 samples
Epoch 1/10
51094/51094 [==============================] - 29s 562us/step - loss: 0.2221 - acc: 0.9318 - val_loss: 0.2439 - val_acc: 0.9291
Epoch 2/10
51094/51094 [==============================] - 29s 573us/step - loss: 0.2229 - acc: 0.9312 - val_loss: 0.2453 - val_acc: 0.9288
Epoch 3/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.2239 - acc: 0.9309 - val_loss: 0.2439 - val_acc: 0.9295
Epoch 4/10
51094/51094 [==============================] - 29s 570us/step - loss: 0.2199 - acc: 0.9325 - val_loss: 0.2449 - val_acc: 0.9294
Epoch 5/10
51094/51094 [==============================] - 29s 569us/step - loss: 0.2184 - acc: 0.9332 - val_loss: 0.2463 - val_acc: 0.9285
Epoch 6/10
51094/51094 [==============================] - 29s 568us/step - loss: 0.2188 - acc: 0.9321 - val_loss: 0.2429 - val_acc: 0.9278
Epoch 7/10
51094/51094 [==============================] - 29s 570us/step - loss: 0.2224 - acc: 0.9312 - val_loss: 0.2

In [41]:
model.evaluate(x_test,y_test)

6835/6835 [==============================] - 1s 109us/step


[0.24254688350371362, 0.9268471105219067]

In [42]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=20,batch_size=32,verbose=1)

Train on 51094 samples, validate on 6798 samples
Epoch 1/20
51094/51094 [==============================] - 29s 561us/step - loss: 0.2142 - acc: 0.9339 - val_loss: 0.2460 - val_acc: 0.9288
Epoch 2/20
51094/51094 [==============================] - 29s 571us/step - loss: 0.2163 - acc: 0.9332 - val_loss: 0.2450 - val_acc: 0.9289
Epoch 3/20
51094/51094 [==============================] - 29s 568us/step - loss: 0.2152 - acc: 0.9339 - val_loss: 0.2444 - val_acc: 0.9306
Epoch 4/20
51094/51094 [==============================] - 29s 567us/step - loss: 0.2146 - acc: 0.9337 - val_loss: 0.2455 - val_acc: 0.9300
Epoch 5/20
51094/51094 [==============================] - 29s 566us/step - loss: 0.2164 - acc: 0.9337 - val_loss: 0.2428 - val_acc: 0.9291
Epoch 6/20
51094/51094 [==============================] - 29s 566us/step - loss: 0.2124 - acc: 0.9344 - val_loss: 0.2436 - val_acc: 0.9295
Epoch 7/20
51094/51094 [==============================] - 29s 567us/step - loss: 0.2120 - acc: 0.9347 - val_loss: 0.2

In [43]:
model.evaluate(x_test,y_test)

6835/6835 [==============================] - 1s 96us/step


[0.24612135037874508, 0.9287490856499243]

In [44]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=20,batch_size=32,verbose=1)

Train on 51094 samples, validate on 6798 samples
Epoch 1/20
51094/51094 [==============================] - 29s 565us/step - loss: 0.2070 - acc: 0.9370 - val_loss: 0.2424 - val_acc: 0.9316
Epoch 2/20
51094/51094 [==============================] - 29s 569us/step - loss: 0.2071 - acc: 0.9359 - val_loss: 0.2429 - val_acc: 0.9313
Epoch 3/20
51094/51094 [==============================] - 29s 568us/step - loss: 0.2042 - acc: 0.9370 - val_loss: 0.2426 - val_acc: 0.9325
Epoch 4/20
51094/51094 [==============================] - 29s 568us/step - loss: 0.2078 - acc: 0.9365 - val_loss: 0.2401 - val_acc: 0.9325
Epoch 5/20
51094/51094 [==============================] - 29s 567us/step - loss: 0.2048 - acc: 0.9368 - val_loss: 0.2422 - val_acc: 0.9323
Epoch 6/20
51094/51094 [==============================] - 29s 567us/step - loss: 0.2040 - acc: 0.9357 - val_loss: 0.2449 - val_acc: 0.9319
Epoch 7/20
51094/51094 [==============================] - 29s 568us/step - loss: 0.2038 - acc: 0.9363 - val_loss: 0.2

In [ ]:
model.evaluate(x_test,y_test)

1600/6835 [======>.......................] - ETA: 0s

In [ ]:
# fpaths = glob.glob(os.path.join('./data_dir/', 'test/audio/*wav'))
# test_data=[]
# fnames=[]
# for path in fpaths:
#     sample_rate, samples = wavfile.read(path)
#     samples = pad_audio(samples)
#     if len(samples) > 16000:
#         n_samples = chop_audio(samples)
#     else :
#         n_samples = [samples]
#     for samples in n_samples:
#         samples=np.asarray(samples).astype('float')
#         S = librosa.feature.melspectrogram(samples, sr=sample_rate, n_mels=128)
#         log_S = librosa.power_to_db(S, ref=np.max)
#         mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
#         fnames.append(path.split('\\')[-1])
#         test_data.append(mfcc)